### UC Berkeley, MICS, W202-Cryptography
### Week 06 Breakout 5
### NIST Dual_EC_DRBG Controversy

#### NIST = National Institute of Standards and Technology
#### NIST SP 800-90A - SP = Special Publication - Recommendation for Random Number Generation Using Deterministic Random Bit Generators
#### Dual_EC_DBRG = a previous standard for an elliptic curve based deterministic random bit generator


As mentioned in our lectures, NIST SP 800-90A specified Dual_EC_DBRG as a standard for an elliptic curve based deterministic random bit generator.  It was withdrawn on June 24, 2015. 

https://csrc.nist.gov/publications/detail/sp/800-90a/archive/2012-01-23

The algorithm specifies a standard elliptic curve prime curve, along with 2 points P and Q. 

There is no mention of how the two points were generated or of their relationship with each other.  If one point is a scalar multiple of the other point, such as P = eQ, then if someone knows e and can intercept one output, they can determine the state and calculate future pseudo random numbers.

As mentioned in our lectures there are unproven allegations that the NSA knew the e and was able to intercept outputs, determine state, and use this to calculate future pseudo random numbers.

In our example below:

* We will take the example of P-256 from the Dual_EC_DRBG specification.  

* We will take the Q from their standard. (In their standard it's actually labelled as P.  They have Q and P labeled in reverse from the lecture notes and formulas - I'm following the lecture notes and formulas in this breakout)

* Since we do NOT know how to find P (we can't solve the elliptic curve discrete logarithm problem), we will generate our own e for the sake of example, and find P = eQ. 

* A, B, p, P, Q would all be publically known.  However, e must be kept secret!

* To make the example more easily understood, we will use points on the elliptic curve for our output pseudo random numbers instead of bit strings.  (It's just a formatting difference - not a mathematical difference).  When the algorithm calls for a "number()" transform, we will use the x coordinate of the point, which is typical in industry implementations.

* We will go a few rounds of generating pseudo random numbers. 

* We will then "intercept" one output pseudo random number, and using our secret e, we will recover the current internal state s, and using the internal state, we can generate the next output pseudo random number. 

* Due to the deterministic (the D in DRBG) nature of the algorithm, we can generate the pseudo random number sequence going forward.

This demonstrates that having access to the secret e and intercepting an output pseudo random number, we can recover the state, and generate the pseudo random number sequence from the point of interception going forward.

In [1]:
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
def my_random_k(p):
    "Given prime p, determine it's bit size, generate a random number 1 bit less than p's size"
    
    upper_limit = (p-2)
    lower_limit = ( (p-2) // 2)
    
    return Integer(randint(lower_limit, upper_limit))

#### NIST SP 800-90A Dual_EC_DRBG Curve P-256

In [4]:
# NIST SP 800-90A Dual_EC_DRBG Curve P-256

p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff

A = -3
B = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

Q_x = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Q_y = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

E = EllipticCurve(GF(p), [A,B])

Q = E(Q_x,Q_y)

In [5]:
my_print_number("A", A)


 A 

decimal: -3 

number of digits: 1 

hex: -3 

number of bits: 2 



In [6]:
my_print_number("B", B)


 B 

decimal: 41,058,363,725,152,142,129,326,129,780,047,268,409,114,441,015,993,725,554,835,256,314,039,467,401,291 

number of digits: 77 

hex: 5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b 

number of bits: 255 



In [7]:
my_print_number("p", p)


 p 

decimal: 115,792,089,210,356,248,762,697,446,949,407,573,530,086,143,415,290,314,195,533,631,308,867,097,853,951 

number of digits: 78 

hex: ffffffff00000001000000000000000000000000ffffffffffffffffffffffff 

number of bits: 256 



In [8]:
my_print_number("Qx", Q_x)


 Qx 

decimal: 48,439,561,293,906,451,759,052,585,252,797,914,202,762,949,526,041,747,995,844,080,717,082,404,635,286 

number of digits: 77 

hex: 6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296 

number of bits: 255 



In [9]:
my_print_number("Qy", Q_y)


 Qy 

decimal: 36,134,250,956,749,795,798,585,127,919,587,881,956,611,106,672,985,015,071,877,198,253,568,414,405,109 

number of digits: 77 

hex: 4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5 

number of bits: 255 



#### NIST SP 800-90A Dual_EC_DRBG Curve P-256 also specifies a point P.  Since we don't know the relationship between P and Q, we will create our own  P based on our own secret e

In [10]:
my_secret_e = 0xde4ef427992637b073ba1c836ac5a149207b21f421a8353605a117d3db185bd6

In [11]:
my_print_number("my secret e", my_secret_e)


 my secret e 

decimal: 

100,552,951,550,212,891,520,808,305,258,392,189,063,659,425,377,548,951,489,238,023,344,229,677,816,790 

number of digits: 78 

hex: de4ef427992637b073ba1c836ac5a149207b21f421a8353605a117d3db185bd6 

number of bits: 256 



In [12]:
P = my_secret_e * Q

In [13]:
my_print_number("Px", Integer(P[0]))


 Px 

decimal: 102,453,850,801,267,965,735,951,010,309,265,686,641,970,952,863,580,121,688,440,687,857,928,876,700,656 

number of digits: 78 

hex: e282d31789682492a32b9a99ec157401f641d780036ca72d1666a8fecd81b3f0 

number of bits: 256 



In [14]:
my_print_number("Py", Integer(P[1]))


 Py 

decimal: 52,180,609,718,048,261,450,537,966,121,058,043,904,451,697,186,656,072,153,357,957,418,669,495,866,738 

number of digits: 77 

hex: 735d2db120c7854bd66cce3b115abde048d7f0b4aa8d07de5ff64583d4b4d572 

number of bits: 255 



#### Create a large seed value of similar size to p which we will use to seed our simple version of EC DRBG.

In [15]:
my_seed = 0xfbb35211360e8bb67b778c0f26137c988161c8bf2b7bc2f5c6c006a93bd0c3db

In [16]:
my_print_number("my seed", my_seed)


 my seed 

decimal: 113,847,357,026,207,571,970,392,002,324,917,075,039,092,242,633,791,678,249,913,775,980,358,021,465,051 

number of digits: 78 

hex: fbb35211360e8bb67b778c0f26137c988161c8bf2b7bc2f5c6c006a93bd0c3db 

number of bits: 256 



#### A simple version of EC DRBG to generate a short sequence of pseudo random numbers.  It also shows the state after each step, but remember state is stored internally and never disclosed.  The pseudo random numbers will be shown as points instead of bit strings to make it easier to follow (it's just a formatting issue - does NOT affect the math)

In [17]:
def my_EC_DRBG(E, P, Q, seed, max_random_numbers):
    "given an elliptic curve prime curve, A, B, p, two points P and Q, generate random numbers and show the state each time"
    
    s = seed
    
    i = 0
    
    while i < max_random_numbers:
        
        i += 1
        
        # we will assume our number function is simply the x value
        # likewise, we will assume our bit string is the point
        
        r = Integer((s * P)[0])
        
        s = Integer((r * P)[0])
        
        ox = Integer((r * Q)[0])
        xy = Integer((r * Q)[1])
        
        print ("\nIteration: " + str(i) + "\n")
        
        print ("    state:\n")
        print ("        " + str(s) + "\n")
        
        print ("    pseudo random number:\n")
        print ("        " + str(ox) + "\n        " + str(xy))


In [18]:
my_EC_DRBG(E, P, Q, my_seed, max_random_numbers=10)


Iteration: 1

    state:

        22582847231691384798543719311246716556578231988974581131471411192103353095979

    pseudo random number:

        95887509679571306593281462914022651400529757548812841831876862188226464808841
        43208131431491273676898070392944689129792220952845186152184960257702810741213

Iteration: 2

    state:

        83565931090699936000172558481132204503914596140062638100984052417049768773670

    pseudo random number:

        72079235288390853398050306380348513314049226978222710083850131474447867105213
        83794819884744576074866305005470738290829578943793254585514142644538899974331

Iteration: 3

    state:

        6066782885071247375958624770119106035299475454453084482196424531233488623033

    pseudo random number:

        40266837503429539539770801352167030626886009514429542143150800340717725334520
        71910300250987652190330345782646279337985174785772547572613008828169228099919



Iteration: 4

    state:

        78628855688711542854691376878903733174722947949524837358524127821382526777679

    pseudo random number:

        37992451932698407326292323676249191818662797731152591198140796177753099984255
        29160257098037928553588076437147622433895935637138889915602534012940588918338

Iteration: 5

    state:

        113266400677542161602942708077741333475354928826806612352602430177493621669733

    pseudo random number:

        115571170669860336291326355791060296124619593043530308010473077044865741301228
        68405120085941136885544795802329690364844406713110247855866790106125424471576

Iteration: 6

    state:

        114275472117032868608863019977709311891928185876329290699619768671774064170098

    pseudo random number:

        50537909385601802490566450261461583424437985512920581838139200978970227414097
        107571893447397199061047266636906490565738026530678804272703012276116604296434



Iteration: 7

    state:

        49280570359575210886514270450413125166020384791816856610788570319742560563538

    pseudo random number:

        76150041792903100209213845815316255050234252955113628839528404278859069588376
        110304334111431836981210299249897603185381477566593504002067685911217045734300

Iteration: 8

    state:

        54483716281585182363721790850074487203634522579146411591892982168007877599783

    pseudo random number:

        38571923860244855550830759760875420175050928248251949378133364764322426142378
        45013602930462807540356511856783106955806930268697721537717585260321437689640

Iteration: 9

    state:

        40035593513129403022977958968834019732214568952299812070000236385056830571252

    pseudo random number:

        86097030428696472288047330941751142864566834126699066376938291059368305676213
        37906452280430623307087050759478872607144986892004381855782651185757853496571



Iteration: 10

    state:

        109742735779892596558400877698656004430734038730775690784251908734984802429919

    pseudo random number:

        70963919164278228573987664347261274037499174422834535665130902144083610244180
        60083794210566639260376721233033913106447121049556232900845121597331434033908


#### Let's assume we know the secret e value, and we just intercepted the pseudo random number from Iteration 5.  Below we will recover the state for Iteration 5, and using that we can generate the next pseudo random number for Iteration 6

In [19]:
# we intercepted randon number 5 and set it to a point on our EC

intercepted_random_number_5 = E(115571170669860336291326355791060296124619593043530308010473077044865741301228,
                                68405120085941136885544795802329690364844406713110247855866790106125424471576)

In [20]:
# using our secret e value, we can recover the state from Iteration 5

# verify our recovered state matched the state for Iteration 5 in the previous printout

recovered_state_5 = Integer((intercepted_random_number_5 * my_secret_e)[0])

recovered_state_5

113266400677542161602942708077741333475354928826806612352602430177493621669733

In [21]:
# now that we have recovered the state for Iteration 5, we can now generate the random number for Iteration 6

# verify our generated pseudo random number below matches the pseudo random number for Iteration 6 in the previous printout

generated_random_number_6 = Integer((recovered_state_5 * P)[0]) * Q

print (generated_random_number_6[0])
print (generated_random_number_6[1])

50537909385601802490566450261461583424437985512920581838139200978970227414097
107571893447397199061047266636906490565738026530678804272703012276116604296434


#### We can now continue generating pseudo random numbers in the sequence